# Ziyao Chen hw4

In [1]:
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict
import nltk
import string
import math
from nltk.stem.porter import *
from sklearn import linear_model
nltk.download('punkt')
from collections import Counter
def parseData(fname):
    for l in urllib.request.urlopen(fname):
        yield eval(l)

[nltk_data] Downloading package punkt to /home/rstudio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [82]:
### Just the first 5000 reviews

print ("Reading data")
data = list(parseData("http://jmcauley.ucsd.edu/cse158/data/beer/beer_50000.json"))[:5000]
print ("done")

Reading data
done


In [3]:
data[:1]

[{'review/appearance': 2.5,
  'beer/style': 'Hefeweizen',
  'review/palate': 1.5,
  'review/taste': 1.5,
  'beer/name': 'Sausa Weizen',
  'review/timeUnix': 1234817823,
  'beer/ABV': 5.0,
  'beer/beerId': '47986',
  'beer/brewerId': '10325',
  'review/timeStruct': {'isdst': 0,
   'mday': 16,
   'hour': 20,
   'min': 57,
   'sec': 3,
   'mon': 2,
   'year': 2009,
   'yday': 47,
   'wday': 0},
  'review/overall': 1.5,
  'review/text': 'A lot of foam. But a lot.\tIn the smell some banana, and then lactic and tart. Not a good start.\tQuite dark orange in color, with a lively carbonation (now visible, under the foam).\tAgain tending to lactic sourness.\tSame for the taste. With some yeast and banana.',
  'user/profileName': 'stcules',
  'review/aroma': 2.0}]

Q1.How many unique bigrams are there amongst all of the reviews? List the 5 most-frequently-occurring
bigrams along with their number of occurrences in the corpus

In [4]:
#read the reviews without capitalization or punctuation
### Ignore capitalization and remove punctuation

wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer()
for d in data:
    r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
    r_token=nltk.word_tokenize(r)
    r_bigrams=list(nltk.bigrams(r_token))
    for w in r_bigrams:
    #w = stemmer.stem(w) # with stemming
        wordCount[w] += 1
print ("There are ",len(wordCount)," unique bigrams.")

There are  182218  unique bigrams.


In [5]:
### Just take the most popular words...

wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
    r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
    r_token=nltk.word_tokenize(r)
    r_bigrams=list(nltk.bigrams(r_token))
    for w in r_bigrams:
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words1 = [x[1] for x in counts[:1000]]

In [6]:
counts[:5]

[(4587, ('with', 'a')),
 (2595, ('in', 'the')),
 (2245, ('of', 'the')),
 (2056, ('is', 'a')),
 (2033, ('on', 'the'))]

# There are 36225 unique bigrams. The 5 most-frequently-occurring bigrams along with their number of occurrences in the corpus are (30695, 'a'), (27569, 'the'), (19512, 'and'), (15935, 'of'), (12623, 'is').

Q2.The code provided performs least squares using the 1000 most common unigrams. Adapt it to use
the 1000 most common bigrams and report the MSE obtained using the new predictor (use bigrams
only, i.e., not unigrams+bigrams) (1 mark). Note that the code performs regularized regression with a
regularization parameter of 1.0.

In [7]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
    r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
    r_token=nltk.word_tokenize(r)
    r_bigrams=list(nltk.bigrams(r_token))
    for w in r_bigrams:
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]

In [8]:
#import numpy
#len(numpy.unique(words))
len(words)

1000

In [87]:
### Sentiment analysis

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
    r_token=nltk.word_tokenize(r)
    r_bigrams=list(nltk.bigrams(r_token))
    for w in r_bigrams:
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in data]
y = [d['review/overall'] for d in data]

In [10]:
#With regularization
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)

In [11]:
type(predictions)

numpy.ndarray

In [12]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y,predictions)

0.3432257072347801

In [13]:
# calculate MSE
zipped = [a - b for a, b in zip(numpy.array(y),numpy.array(predictions))]
myarray = numpy.asarray(zipped)
mse = numpy.mean(myarray**2)

In [14]:
mse

0.3432257072347801

# The MSE obtained using the new predictor is 0.3432257072347801.

Q3.What is the inverse document frequency of the words `foam', `smell', `banana', `lactic', and `tart'? What are their tf-idf scores in the rst review (using log base 10)

In [15]:
document=[]
for d in data:
    r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
    document.append(r.split())
    #document.append(r)

In [16]:
def computeIDF(wordlist,document):
    idf_cal= defaultdict(set)
    idfDict= defaultdict(set)
    N=len(document)

    for word in wordlist:
        i=0
        for d in document:
            if word in d:
                i=i+1
        idf_cal[word]=i
                
    for word, value in idf_cal.items():
        idfDict[word]=math.log10(N/float(value))
        
    return idfDict

In [17]:
wordlist=['foam','smell','banana','lactic','tart']
idf_q3=computeIDF(wordlist,document)

# The inverse document frequency of the words are shown as below.

In [18]:
idf_q3

defaultdict(set,
            {'foam': 1.1378686206869628,
             'smell': 0.5379016188648442,
             'banana': 1.6777807052660807,
             'lactic': 2.9208187539523753,
             'tart': 1.8068754016455384})

In [19]:
# calculate tf for the first review
def computeTF(wordlist,document):
    tfDict= defaultdict(set)
    
    for word in wordlist:
        count=0
        for d in document:
                if word == d:
                    count+=1
        tfDict[word]=count
        
    return tfDict

In [20]:
document0=document[0]
tf_r1=computeTF(wordlist,document0)

In [21]:
tf_r1

defaultdict(set, {'foam': 2, 'smell': 1, 'banana': 2, 'lactic': 2, 'tart': 1})

In [22]:
# function to calculate tf-idf 
def computeTFIDF(wordlist,tf,idf):
    tfidf=defaultdict(set)
    for word in wordlist:
        tfidf[word]=tf[word]*idf[word]
    return tfidf

In [23]:
tfidf_r1=computeTFIDF(wordlist,tf_r1,idf_q3)
tfidf_r1

defaultdict(set,
            {'foam': 2.2757372413739256,
             'smell': 0.5379016188648442,
             'banana': 3.3555614105321614,
             'lactic': 5.841637507904751,
             'tart': 1.8068754016455384})

# The tfidf scores are shown as above.

# Q4.What is the cosine similarity between the first and the second review in terms of their tf-idf representations

In [24]:
# define function to calculat tf-idf
def computeTF_IDF(wordlist,documentset):
    tfDict= defaultdict(set)
    idf_cal= defaultdict(set)
    idfDict= defaultdict(set)
    tfidf=defaultdict(set)
    N=len(document)
    
    # tf
    for word in wordlist:
        count=0
        for d in documentset:
                if word == d:
                    count+=1
        tfDict[word]=count
        
    #idf    
        i=0
        for d in document:
            if word in d:
                i=i+1
        idf_cal[word]=i
        
    for word, value in idf_cal.items():
        idfDict[word]=math.log10(N/float(value))   
    
    #tf-idf
    for word in wordlist:
        tfidf[word]=tfDict[word]*idfDict[word]
    
    return tfidf

In [25]:
# find common words between the first and second review
r1=document[0]
r2=document[1]
wordlist_q4 = set(r1)&set(r2)

In [26]:
type(wordlist_q4)

set

In [27]:
# get the tf-idf score of reviews
tfidf_q4_r1=computeTF_IDF(wordlist_q4,r1)
tfidf_q4_r2=computeTF_IDF(wordlist_q4,r2)

In [28]:
tfidf_q4_r1

defaultdict(set,
            {'foam': 2.2757372413739256,
             'smell': 0.5379016188648442,
             'in': 0.3496672554869006,
             'dark': 0.5512936800949202,
             'taste': 0.2914091548496563,
             'not': 0.2823294969977378,
             'the': 0.0867293605886536,
             'color': 0.46067293646062496,
             'with': 0.12436263060328803,
             'of': 0.05158703422139899,
             'but': 0.1663430031071276,
             'again': 0.8787685448503786,
             'a': 0.024492340235155226,
             'and': 0.0979177801021508})

In [29]:
# turn set in to lists
list1=[]
list2=[]
for key,value in tfidf_q4_r1.items():
    list1.append(value)
    
for key,value in tfidf_q4_r2.items():
    list2.append(value)

 calculate cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([list1], [list2])

In [30]:
tfidf1=computeTF_IDF(r1,r1)
sum1=sum(tfidf1[x]**2 for x in tfidf1.keys())
tfidf2=computeTF_IDF(r2,r2)
sum2=sum(tfidf2[x]**2 for x in tfidf2.keys())

In [31]:
import math
sim=0
for i in range(len(list1)):
    sim+=list1[i]*list2[i]/(math.sqrt(sum1)*math.sqrt(sum2))

In [32]:
sim

0.06588193974744379

# The cosine similarity between the first and the second review is 0.0658819397474438.

Q5.Which other review has the highest cosine similarity compared to the rst review (provide the beerId
and proleName, or the text of the review)

In [33]:
# get a idf dic
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
  r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
  for w in r.split():
    wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
words = [x[1] for x in counts]

In [34]:
idf_q5=computeIDF(words,document)

In [40]:
# define function to calculat tf-idf
def computeTF_IDF5(wordlist,documentset):
    tfDict= defaultdict(set)
    tfidf=defaultdict(set)
    
    # tf
    for word in wordlist:
        count=0
        for d in documentset:
                if word == d:
                    count+=1
        tfDict[word]=count
         
        tfidf[word]=tfDict[word]*idf_q5[word]
    
    return tfidf

In [41]:
documentget=defaultdict(set)
n=0
for d in document:
    documentget[n]=d
    n+=1

In [42]:
cosdic=[]
for i in range(1,5001):
    #calculate tf-idf
    review=documentget[i]
    wordlist_q5 = set(r1)&set(review)
    tfidf_q5_review=computeTF_IDF5(wordlist_q5,review)
    tfidf_q5_r1=computeTF_IDF5(wordlist_q5,r1)
    
    #calculate cosine
    tfidfb=computeTF_IDF5(review,review)
    sum2=sum(tfidfb[x]**2 for x in tfidfb.keys())
    if sum2==0:
        cosdic.append(0)
    else:
        cosine_similarity=sum([tfidf_q5_review[x]*tfidf_q5_r1[x] for x in wordlist_q5])/(math.sqrt(sum1)*math.sqrt(sum2))
        cosdic.append(cosine_similarity)

In [43]:
cosdic.index(max(cosdic))

2342

In [110]:
max(cosdic)

0.2968679537499198

In [44]:
data[2343]

{'review/appearance': 4.0,
 'beer/style': 'Saison / Farmhouse Ale',
 'review/palate': 4.0,
 'review/taste': 3.5,
 'beer/name': 'Her Majesty 2011',
 'review/timeUnix': 1317979248,
 'user/gender': 'Male',
 'user/birthdayRaw': 'Mar 14, 1977',
 'beer/ABV': 8.0,
 'beer/beerId': '72146',
 'user/birthdayUnix': 227174400,
 'beer/brewerId': '23246',
 'review/timeStruct': {'isdst': 0,
  'mday': 7,
  'hour': 9,
  'min': 20,
  'sec': 48,
  'mon': 10,
  'year': 2011,
  'yday': 280,
  'wday': 4},
 'user/ageInSeconds': 1191161047,
 'review/overall': 4.0,
 'review/text': "750mL bottle thanks to Chris@Slowbeer. Poured into a Lost Abbey stemmed tulip.\t\tGolden orange, close to translucent (on the first pour at least), capped by a sizable white, typically Belgian-looking head. Good lacing.\t\tQuite strong lactic notes and a sharp organic funk. Pungent stuff. Underneath is bitter citrus pith, floral spice and a hint of sweet esters. In your face with a lot going on. Only issue is the lactic character ver

# 'beer/beerId': '72146' has the highest cosine similarity compared to the first review.

Q6.Adapt the original model that uses the 1000 most common unigrams, but replace the features with their
1000-dimensional tf-idf representations, and report the MSE obtained with the new model.

In [100]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
  r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
  for w in r.split():
    wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words1 = [x[1] for x in counts[:1000]]

In [102]:
X = [computeTF_IDF5(words1,d) for d in document]

In [103]:
# methond to convert dict to list
final=[]
for i in range(0,5000):
    x_try=[]
    for y in X[i]:
        x_try.append(X[i][y])
    final.append(x_try)

In [105]:
y = [d['review/overall'] for d in data]
y[:10]

[1.5, 3.0, 3.0, 3.0, 4.0, 3.0, 3.5, 3.0, 4.0, 4.5]

In [108]:
#With regularization
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(final, y)
theta = clf.coef_
predictions = clf.predict(final)

In [109]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y,predictions)

1.1349391271538383